## Building a RAG System with LlamaIndex and Groq API

Retrieval-Augmented Generation (RAG) enhances language models by providing them with relevant documents to improve their answers. In this guide, you'll learn how to build a simple RAG pipeline using:

LlamaIndex for document indexing and retrieval

Groq API for fast inference with LLaMA3 or similar models



### Setup

Install the required Python packages:

In [1]:
%pip install -Uq 'llama-index'
%pip install -Uq llama-index-llms-openai
%pip install -Uq llama-index-llms-groq

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'llama-index'": Expected package name at the start of dependency specifier
    'llama-index'
    ^


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -Uq python-dotenv

import os
from dotenv import load_dotenv

if os.path.exists('../.env'):
    load_dotenv('../.env')

Note: you may need to restart the kernel to use updated packages.


Groq supports OpenAI-compatible endpoints, so you can use it just like OpenAI's API. Set your API key like this:


In [ ]:
# we will use Groq models to make this free to use. but we still need an API key
import os
import getpass

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "API KEY HERE"

In [4]:
# we will be using pprint for logging
from pprint import pprint

In [5]:
from llama_index.llms.groq import Groq

llm = Groq(model="llama3-70b-8192")

C:\Users\TAURO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### RAG 

Install packages necesary to translate data into readable context:

In [12]:
%pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   -------------- ------------------------- 6.0/16.6 MB 36.2 MB/s eta 0:00:01
   ---------------------------------- ----- 14.4/16.6 MB 39.7 MB/s eta 0:00:01
   ---------------------------------------- 16.6/16.6 MB 37.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Install embeddings and setup base llm and base embeded model of llama into groq to avoid problems:

In [ ]:
%pip install llama-index llama-index-embeddings-openai groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement llama-index-embeddings-sentence-transformers (from versions: none)
ERROR: No matching distribution found for llama-index-embeddings-sentence-transformers


In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Use HuggingFace Embeddings (local / built-in)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Tell LlamaIndex to use them everywhere by default
Settings.llm = llm
Settings.embed_model = embed_model

C:\Users\TAURO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TAURO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\Local\llama_index\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/g

This script defines a function to load PDF files from a folder, extract their text using PyMuPDF (fitz), and convert them into LlamaIndex Document objects for later indexing.

In [15]:
import os
import fitz  # PyMuPDF
from llama_index.core import VectorStoreIndex, Document, ServiceContext

def load_pdfs_from_folder(folder_path):
    docs = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            path = os.path.join(folder_path, filename)
            print(f"Reading {filename}...")
            try:
                pdf_doc = fitz.open(path)
                text = "\n".join([page.get_text() for page in pdf_doc])
                if not text.strip():
                    raise ValueError("Empty text, maybe scanned.")
                docs.append(Document(text=text, metadata={"filename": filename}))
            except Exception as e:
                print(f"Failed to read {filename}: {e}")
    
    return docs


Load and Index Your Documents:

In [16]:
docs = load_pdfs_from_folder("data")
index = VectorStoreIndex.from_documents(docs)

Reading colombia91.pdf...


Create a Query Engine

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3)

Ask Questions!

In [ ]:
response = query_engine.query("¿Cuales son los derechos fundamentales de los ciudadanos?")
print(response)

Los derechos fundamentales de los ciudadanos, según el texto, incluyen:

* El derecho a la vida (Artículo 11)
* La prohibición de la desaparición forzada, torturas, tratos o penas crueles, inhumanos o degradantes (Artículo 12)
* La igualdad ante la ley y la protección contra la discriminación (Artículo 13)
* El reconocimiento de la personalidad jurídica (Artículo 14)
* El derecho a la intimidad personal y familiar, y a la protección de la correspondencia y demás formas de comunicación privada (Artículo 15)
* El libre desarrollo de la personalidad sin más limitaciones que las que imponen los derechos de los demás y el orden jurídico (Artículo 16)
* La prohibición de la esclavitud, la servidumbre y la trata de seres humanos en todas sus formas (Artículo 17)
* La libertad de conciencia y de cultos (Artículos 18 y 19)
* La libertad de expresión y de información (Artículo 20)

Estos son algunos de los derechos fundamentales establecidos en la Constitución.
